In [44]:
import pandas as pd
import numpy as np
from pymoo.algorithms.unsga3 import UNSGA3
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination

from combined_problem import CombinedProblem
from notebooks.optimization_problems.constraints import Requirements

In [45]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "telesat_post_processed.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)
instances_df = instances_500km_df

In [46]:
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('../dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df[['isend', 'isend_200MHz', 'isend_300MHz']].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 120e9
requirements.max_latency = 60.0 * 60.0 * 1.5

problem = CombinedProblem(instances_df, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3]))

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 100, 'n_offsprings': 25, 'n_max_gen': 5000}

settings = {}

# U-NSGA-III
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_reference_directions, get_visualization, get_termination

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=64)
print("# dirs: %d" % len(ref_dirs))

#get_visualization("scatter").add(ref_dirs).show()
settings['UNSGA-III'] = {
    'label': 'unsga3',
    'algorithm': UNSGA3(
        pop_size=int(np.ceil(len(ref_dirs) / 100) * 100),
        n_offsprings=int(np.ceil(len(ref_dirs) / 100) * 100 / 4),
        sampling=sampling,#InitialSampling(),
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    # 'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
    #                                                   n_last=30,
    #                                                   nth_gen=5,
    #                                                   n_max_gen=algo_settings['n_max_gen'],
    #                                                   n_max_evals=None)
    'termination': get_termination("time", "01:00:00")
}

setting = settings['UNSGA-III']

# dirs: 2145


In [47]:
termination = setting['termination']
algorithm = setting['algorithm']

n_gen_inc = 1000

In [48]:
from os import path

file_name = 'Telesat_%s_%d_%d.pkl' % (setting['label'], algorithm.pop_size, algorithm.n_offsprings)

if path.exists(file_name):
    problem, setting, res = pickle.load(open(file_name, 'rb'))
    algorithm = res.algorithm
    algorithm.has_terminated = False
    n_gen = res.algorithm.n_gen
else:
    n_gen = 0

In [49]:
n_gen = n_gen + n_gen_inc
print(n_gen)

res = minimize(problem,
               algorithm,
               ('n_gen', n_gen),
               #termination=termination,
               seed=1,
               #save_history=True,
               verbose=True,
               copy_algorithm=False,
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

pickle.dump((problem, setting, res), open(file_name, 'wb'))

1000
    1 |    2200 |  2.05400E+03 |  2.64691E+03 |       1 |  3.61640E+12 |        ideal
    2 |    2750 |  2.05400E+03 |  2.55547E+03 |       1 |  0.00000E+00 |            f
    3 |    3300 |  1.97500E+03 |  2.49063E+03 |       1 |  0.00000E+00 |            f
    4 |    3850 |  1.90100E+03 |  2.43432E+03 |       1 |  0.00000E+00 |            f
    5 |    4400 |  1.86000E+03 |  2.38435E+03 |       1 |  0.00000E+00 |            f
    6 |    4950 |  1.82600E+03 |  2.33797E+03 |       1 |  0.00000E+00 |            f
    7 |    5500 |  1.82600E+03 |  2.29345E+03 |       1 |  0.00000E+00 |            f
    8 |    6050 |  1.68800E+03 |  2.24950E+03 |       1 |  0.00000E+00 |            f
    9 |    6600 |  1.68800E+03 |  2.20580E+03 |       1 |  0.00000E+00 |            f
   10 |    7150 |  1.68800E+03 |  2.16295E+03 |       1 |  0.00000E+00 |            f
   11 |    7700 |  1.68800E+03 |  2.12076E+03 |       1 |  0.00000E+00 |            f
   12 |    8250 |  1.67700E+03 |  2.07834E+03 |  